# EDA Solutions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
sns.set_style('whitegrid')
%matplotlib inline

## 1️⃣ Used Bike Details Dataset
Load the dataset first:

In [ ]:
# ⬇️ Update the path to your CSV
bike_df = pd.read_csv('bike_details.csv')
print('Rows:', bike_df.shape[0], '| Columns:', bike_df.shape[1])
bike_df.head()

🔎 **Q1. Range of selling prices**

In [ ]:
price_min, price_max = bike_df['selling_price'].agg(['min','max'])
print(f'₹{price_min:,} – ₹{price_max:,}')

🔸 **Q2. Median selling price**

In [ ]:
median_price = bike_df['selling_price'].median()
print(f'Median selling price: ₹{median_price:,}')

🔸 **Q3. Most common seller type**

In [ ]:
most_common = bike_df['seller_type'].mode()[0]
print('Most common seller type:', most_common)

🔸 **Q4. Bikes driven > 50 000 km**

In [ ]:
over_50k = (bike_df['km_driven'] > 50000).sum()
print('Count:', over_50k)

🔸 **Q5. Average `km_driven` by ownership type**

In [ ]:
avg_km_by_owner = bike_df.groupby('owner')['km_driven'].mean().sort_values(ascending=False)
avg_km_by_owner

🔸 **Q6. Proportion of bikes from ≤ 2015**

In [ ]:
prop_2015_older = (bike_df['year'] <= 2015).mean()
print(f'{prop_2015_older:.1%} of bikes are from 2015 or older')

🔸 **Q7. Missing‑value overview**

In [ ]:
missing = bike_df.isna().sum()
missing

🔸 **Q8. Highest `ex_showroom_price` & bike**

In [ ]:
idx = bike_df['ex_showroom_price'].idxmax()
highest_row = bike_df.loc[idx, ['name','ex_showroom_price']]
highest_row

🔸 **Q9. Total bikes by seller type**

In [ ]:
bike_df['seller_type'].value_counts()

🔸 **Q10. Relationship between `selling_price` & `km_driven` for *first‑owner* bikes**

In [ ]:
first_owner = bike_df[bike_df['owner'].str.contains('1st', case=False)]
plt.figure()
plt.scatter(first_owner['km_driven'], first_owner['selling_price'], alpha=0.5)
plt.title('Price vs. KM (1st Owner)')
plt.xlabel('KM Driven')
plt.ylabel('Selling Price (₹)')
plt.show()

🔸 **Q11. Remove `km_driven` outliers using IQR**

In [ ]:
Q1 = bike_df['km_driven'].quantile(0.25)
Q3 = bike_df['km_driven'].quantile(0.75)
IQR = Q3 - Q1
lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
bike_df_clean = bike_df[(bike_df['km_driven'] >= lower) & (bike_df['km_driven'] <= upper)]
print('Remaining rows after outlier removal:', bike_df_clean.shape[0])

🔸 **Q12. Year vs. Selling Price (scatter)**

In [ ]:
plt.figure()
plt.scatter(bike_df['year'], bike_df['selling_price'], alpha=0.4)
plt.title('Year vs. Selling Price')
plt.xlabel('Year')
plt.ylabel('Selling Price (₹)')
plt.show()

🔸 **Q13. Average depreciation by age**

In [ ]:
current_year = date.today().year
bike_df['age'] = current_year - bike_df['year']
dep_by_age = bike_df.groupby('age')['selling_price'].mean().sort_index()
dep_by_age.head()

🔸 **Q14. Bikes priced significantly above annual average**

In [ ]:
# Computes z‑score for each bike vs its year average
bike_df['year_avg'] = bike_df.groupby('year')['selling_price'].transform('mean')
bike_df['z_score'] = (bike_df['selling_price'] - bike_df['year_avg'])/bike_df.groupby('year')['selling_price'].transform('std')
above_2sd = bike_df[bike_df['z_score'] > 2][['name','year','selling_price','z_score']]
above_2sd.head()

🔸 **Q15. Correlation heatmap (numeric cols)**

In [ ]:
plt.figure(figsize=(6,4))
cor = bike_df[['selling_price','km_driven','year','ex_showroom_price']].corr()
sns.heatmap(cor, annot=True, cmap='Blues')
plt.title('Correlation Matrix')
plt.show()


---
## 2️⃣ Used Car Sales Dataset

Load the dataset:

In [ ]:
# ⬇️ Update the path
car_df = pd.read_csv('car_sales.csv')
print('Rows:', car_df.shape[0])
car_df.head()

🔎 **Q1. Average selling price per dealer & comparison**

In [ ]:
avg_price_dealer = car_df.groupby('Dealer_Name')['Price ($)'].mean().sort_values(ascending=False)
avg_price_dealer.head()

🔸 **Q2. Brand with highest price variation**

In [ ]:
variation = car_df.groupby('Company')['Price ($)'].agg(np.var).sort_values(ascending=False)
variation.head()

🔸 **Q3. Price distribution by transmission (box plot)**

In [ ]:
plt.figure()
sns.boxplot(data=car_df, x='Transmission', y='Price ($)')
plt.title('Price vs Transmission')
plt.show()

🔸 **Q4. Price distribution across regions**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(data=car_df, x='Dealer_Region', y='Price ($)')
plt.show()

🔸 **Q5. Distribution of cars based on body styles**

In [ ]:
car_df['Body Style'].value_counts()

🔸 **Q6. Avg price by customer gender & income**

In [ ]:
car_df['Income Bracket'] = pd.cut(car_df['Annual Income'], bins=[0,30000,60000,90000, np.inf], labels=['<30k','30‑60k','60‑90k','90k+'])
car_gender_income = car_df.groupby(['Gender','Income Bracket'])['Price ($)'].mean().unstack()
car_gender_income

🔸 **Q7. Price & volume by region**

In [ ]:
avg_price_region = car_df.groupby('Dealer_Region')['Price ($)'].mean()
count_region = car_df['Dealer_Region'].value_counts()
pd.concat([avg_price_region, count_region], axis=1).rename(columns={'Price ($)':'Avg Price','Dealer_Region':'Count'})

🔸 **Q8. Price difference across engine sizes**

In [ ]:
avg_price_engine = car_df.groupby('Engine')['Price ($)'].mean().sort_values(ascending=False)
avg_price_engine

🔸 **Q9. Price vs customer income bracket**

In [ ]:
plt.figure()
sns.boxplot(data=car_df, x='Income Bracket', y='Price ($)')
plt.show()

🔸 **Q10. Top‑5 car models by sales & price distribution**

In [ ]:
top_models = car_df['Model'].value_counts().head(5).index
plt.figure()
sns.boxplot(data=car_df[car_df['Model'].isin(top_models)], x='Model', y='Price ($)')
plt.show()

🔸 **Q11. Price vs engine size across colors**

In [ ]:
plt.figure(figsize=(8,4))
sns.scatterplot(data=car_df, x='Engine', y='Price ($)', hue='Color', alpha=0.6)
plt.show()

🔸 **Q12. Seasonal trend in sales (by month)**

In [ ]:
car_df['Month'] = pd.to_datetime(car_df['Date']).dt.month
sales_by_month = car_df.groupby('Month')['Car_id'].count()
sales_by_month.plot(kind='bar', title='Cars Sold by Month');

🔸 **Q13. Price distribution for Body Style × Transmission**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(data=car_df, x='Body Style', y='Price ($)', hue='Transmission')
plt.xticks(rotation=45)
plt.show()

🔸 **Q14. Correlation: price, engine size & annual income**

In [ ]:
corr_cols = car_df[['Price ($)','Annual Income']].copy()
# Assuming engine is numeric; if not, map to size
corr_cols['EngineSize'] = car_df['Engine'].str.extract('(\d+)').astype(float)
sns.pairplot(corr_cols)
plt.show()

🔸 **Q15. Avg price across models & engine types**

In [ ]:
avg_model_engine = car_df.groupby(['Model','Engine'])['Price ($)'].mean().sort_values(ascending=False).head(10)
avg_model_engine


---
## 3️⃣ Amazon Product Sales Dataset

In [ ]:
amazon_df = pd.read_csv('amazon_sales.csv')
amazon_df.head()

🔎 **Q1. Average rating per category**

In [ ]:
avg_rating = amazon_df.groupby('category')['rating'].mean().sort_values(ascending=False)
avg_rating.head()

🔸 **Q2. Top products by rating count per category**

In [ ]:
top_count = amazon_df.sort_values(['category','rating_count'], ascending=[True,False]).groupby('category').head(3)[['product_name','rating_count']]
top_count

🔸 **Q3. Distribution of discounted vs actual prices**

In [ ]:
plt.figure()
sns.histplot(amazon_df['discounted_price'], label='Discounted', kde=True, alpha=0.5)
sns.histplot(amazon_df['actual_price'], label='Actual', kde=True, alpha=0.5)
plt.legend()
plt.show()

🔸 **Q4. Average discount % across categories**

In [ ]:
avg_disc = amazon_df.groupby('category')['discount_percentage'].mean().sort_values(ascending=False)
avg_disc.head()

🔸 **Q5. Most popular product names**

In [ ]:
amazon_df['product_name'].value_counts().head()

🔸 **Q6. Most common product keywords**

In [ ]:
from collections import Counter
keywords = amazon_df['product_name'].str.lower().str.split().explode()
common_keywords = Counter(keywords).most_common(10)
common_keywords

🔸 **Q7. Most popular review titles**

In [ ]:
amazon_df['review_title'].value_counts().head()

🔸 **Q8. Correlation: discounted_price vs rating**

In [ ]:
corr = amazon_df[['discounted_price','rating']].corr().iloc[0,1]
print('Correlation:', corr)

🔸 **Q9. Top‑5 categories by highest ratings**

In [ ]:
top_cat = amazon_df.groupby('category')['rating'].mean().sort_values(ascending=False).head(5)
top_cat

🔸 **Q10. Improvement opportunities**

In [ ]:
display('👇 Potential optimisation areas:',
        '* Categories with high discounts but low ratings could indicate quality issues',
        '* Products with many reviews but mediocre ratings might benefit from feedback analysis',
        '* Low discount % and low sales volume categories could be candidates for promotional campaigns')


---
## 4️⃣ Spotify Hip‑hop Tracks Dataset

In [ ]:
spotify_df = pd.read_csv('spotify_hiphop.csv')
spotify_df.head()

🔎 **Q1. Handle nulls & duplicates**

In [ ]:
# Drop duplicate rows
spotify_df = spotify_df.drop_duplicates()
# Fill or drop nulls
spotify_df = spotify_df.dropna()
spotify_df.isna().sum()

🔸 **Q2. Popularity distribution (histogram)**

In [ ]:
plt.figure()
plt.hist(spotify_df['Popularity'], bins=20)
plt.title('Popularity Distribution')
plt.xlabel('Popularity')
plt.ylabel('Frequency')
plt.show()

🔸 **Q3. Popularity vs duration (scatter)**

In [ ]:
plt.figure()
plt.scatter(spotify_df['Duration (ms)'], spotify_df['Popularity'], alpha=0.4)
plt.xlabel('Duration (ms)')
plt.ylabel('Popularity')
plt.title('Popularity vs Duration')
plt.show()

🔸 **Q4. Artist track counts (countplot)**

In [ ]:
plt.figure(figsize=(8,4))
# Show top 10 for clarity
sns.countplot(data=spotify_df, y='Artist', order=spotify_df['Artist'].value_counts().head(10).index)
plt.title('Tracks per Artist (top 10)')
plt.show()

🔸 **Q5. 5 least popular tracks**

In [ ]:
spotify_df.nsmallest(5, 'Popularity')[['Artist','Track Name','Popularity']]

🔸 **Q6. Avg popularity among top‑5 artists**

In [ ]:
top5_artists = spotify_df['Artist'].value_counts().head(5).index
avg_pop = spotify_df[spotify_df['Artist'].isin(top5_artists)].groupby('Artist')['Popularity'].mean()
avg_pop

🔸 **Q7. Most popular track per top‑5 artist**

In [ ]:
most_pop_tracks = spotify_df[spotify_df['Artist'].isin(top5_artists)].sort_values(['Artist','Popularity'], ascending=[True,False]).groupby('Artist').head(1)[['Artist','Track Name','Popularity']]
most_pop_tracks

🔸 **Q8. Pair plot of numeric vars**

In [ ]:
sns.pairplot(spotify_df[['Popularity','Duration (ms)']])
plt.show()

🔸 **Q9. Duration variation across artists (box plot)**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(data=spotify_df, x='Artist', y='Duration (ms)', order=top5_artists)
plt.xticks(rotation=45)
plt.show()

🔸 **Q10. Popularity distribution per artist (violin)**

In [ ]:
plt.figure(figsize=(8,4))
sns.violinplot(data=spotify_df[spotify_df['Artist'].isin(top5_artists)], x='Artist', y='Popularity')
plt.show()